In [1]:
from langchain.utilities import SerpAPIWrapper
SERPAPI_API_KEY="9db0c9c4de64a99c8f62582922b7cc21afaaaf354a6788f66a6fe2e107b4a74d"


In [11]:
!pip -q install duckduckgo-search

In [1]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [9]:
print(search.run("One America News Suggests ivermectin is an effective treatment for COVID-19"))

Alex Kasprak Published Aug 15, 2023 Image Via Twitter Claim: In August 2023, the U.S. Food and Drug Administration reversed course and admitted that doctors would be allowed to prescribe... Ivermectin Treatment for Covid-19 To the Editor Reis et al. (May 5 issue) 1 present the results of the TOGETHER platform trial, a large trial of ivermectin in outpatients with Covid-19. The FDA has not changed its position on ivermectin, and experts maintain it is not an effective treatment for COVID-19. The agency has not authorized or approved the drug for use in preventing or ... Answer From Daniel C. DeSimone, M.D. The FDA has approved an antiviral drug called remdesivir (Veklury) to treat COVID-19 in adults and children who are age 12 and older. Remdesivir may be prescribed for people who are hospitalized with COVID-19 and need supplemental oxygen or have a higher risk of serious illness. Ivermectin has been shown to inhibit replication of SARS-CoV-2 in cell cultures. 3 However, pharmacokinetic

In [4]:
from langchain.utilities import SerpAPIWrapper

# Your SerpAPI key
SERPAPI_API_KEY = "9db0c9c4de64a99c8f62582922b7cc21afaaaf354a6788f66a6fe2e107b4a74d"

# Initialize the SerpAPIWrapper
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)

# Define your search parameters
search_params = {
    "engine": "google",  # Specify the search engine (e.g., google, bing, etc.)
    "q": "OpenAI",       # Your query
    'gl': 'us',          # Specify the country
    'hl': 'en',          # Specify the language
    # Add other parameters as needed
}

# Perform the search
results = search.run(search_params)

# Process the results
# The structure of 'results' will depend on the type of search and the engine used
print(results)


[{'title': 'Amazon announces Q, an AI chatbot for businesses', 'link': 'https://www.cnbc.com/2023/11/28/amazon-announces-q-an-ai-chatbot-for-businesses.html', 'source': 'CNBC', 'date': '1 day ago', 'thumbnail': 'https://serpapi.com/searches/656879a8b5961edafc902026/images/e17d4bf68c1eb2a47685e932563123b755fb692095f4c45c.jpeg'}, {'title': 'AWS-Announces-Four-New-Capabilities-for-AWS-Supply-Chain', 'link': 'https://press.aboutamazon.com/2023/11/aws-announces-four-new-capabilities-for-aws-supply-chain', 'source': 'Amazon Press Releases', 'date': '1 day ago', 'thumbnail': 'https://serpapi.com/searches/656879a8b5961edafc902026/images/e17d4bf68c1eb2a43ac8f3a44a4b13d30f3d3525427bb4b4.jpeg'}, {'title': 'Amazon Introduces Q, an A.I. Chatbot for Companies', 'link': 'https://www.nytimes.com/2023/11/28/technology/amazon-ai-chatbot-q.html', 'source': 'The New York Times', 'date': '1 day ago', 'thumbnail': 'https://serpapi.com/searches/656879a8b5961edafc902026/images/e17d4bf68c1eb2a4323f6e35569fe217

In [10]:
# Import things that are needed generically
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.utilities import SerpAPIWrapper

llm = ChatOpenAI(temperature=0)

search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
# llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events",
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

# from pydantic import BaseModel, Field


# class CalculatorInput(BaseModel):
#     question: str = Field()


# tools.append(
#     Tool.from_function(
#         func=llm_math_chain.run,
#         name="Calculator",
#         description="useful for when you need to answer questions about math",
#         args_schema=CalculatorInput,
#         # coroutine= ... <- you can specify an async method if desired as well
#     )
# )

# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

agent.run(
    "Detailed description of 9/11"
)



> Entering new AgentExecutor chain...
I should search for a detailed description of 9/11 to provide an accurate answer.
Action: Search
Action Input: "detailed description of 9/11"
Observation: On the morning of 11 September 2001, 19 Al Qaeda terrorists hijacked four commercial passenger planes in the United States. Two planes were flown into the Twin Towers of the World Trade Center in New York City, causing both towers to collapse. A third plane was crashed into the Pentagon, just outside Washington, DC.
Thought:This is a brief summary of the events of 9/11, but I should provide more details.
Action: Search
Action Input: "detailed timeline of 9/11"
Observation: ['Devastation at the World Trade Center site in New York City in the aftermath of the 11 September 2001 terrorist attack. National Archives ...', 'Two planes, hijacked by Islamic jihadists vowing death to all Americans, plowed into both towers at the World Trade Center in New York. Another plane was flown ...', 'As no other e

'On the morning of September 11, 2001, 19 Al Qaeda terrorists hijacked four commercial passenger planes in the United States. Two planes were flown into the Twin Towers of the World Trade Center in New York City, causing both towers to collapse. Another plane was crashed into the Pentagon, just outside Washington, DC.'

In [15]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

schema = {
    "properties": {
        "fact_label": {
            "type": "string",
            "enum": ['True', 'Mostly True', 'Half True', "Mostly False", "False", "Pants on Fire"],
            "description": """Please classify the given statement into one of the following categories:
    True - The statement is accurate and there is nothing significant missing.
    Mostly True - The statement is accurate but needs clarification or additional information.
    Half True - The statement is partially accurate but leaves out important details or takes things out of context.
    Mostly False - The statement contains an element of truth but ignores critical facts that would give a different impression.
    False - The statement is not accurate.
    Pants on Fire - The statement is not accurate and makes a ridiculous claim including a conspiracy.
    The statement will have the following format:
    statement|author|date
    e.g.:'Last year alone, natural disasters in America caused $178 billion in damages.|Barack Obama|2013-02-13'"""
        },
    },
    "required": ["fact_label"],
}
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_tagging_chain(schema, llm)
inp = "Me and Osamas Family are quite close.|George Bush|2001-09-11"
chain.run(inp)



{'fact_label': 'True'}

In [29]:
from langchain_core.prompts import ChatPromptTemplate
schema = {
        "properties": {
            "fact_label": {
                "type": "string",
                "enum": ['True', 'Mostly True', 'Half True', "Mostly False", "False", "Pants on Fire"],
                # "description": "classify a statement based on its truthfulness"
            },
        },
        "required": ["fact_label"],
    }
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
prompt = """Please classify the given statement into one of the following categories:
        True - The statement is accurate and there is nothing significant missing.
        Mostly True - The statement is accurate but needs clarification or additional information.
        Half True - The statement is partially accurate but leaves out important details or takes things out of context.
        Mostly False - The statement contains an element of truth but ignores critical facts that would give a different impression.
        False - The statement is not accurate.
        Pants on Fire - The statement is not accurate and makes a ridiculous claim including a conspiracy.
        The statement will have the following format:
        statement|author|date
        e.g.:'Last year alone, natural disasters in America caused $178 billion in damages.|Barack Obama|2013-02-13'"""
chain = create_tagging_chain(schema=schema, llm=llm, prompt=ChatPromptTemplate.from_template(prompt))
chain.run({"schema":"Last 9/11 year alone, natural disasters in America caused $178 billion in damages.|Barack Obama|2013-02-13", "WTF": "fuck me"})

{'fact_label': 'True'}

In [12]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI

from classifier_new import fact_check

# Parameters
file_path = "../../../data/politifact/sample.csv"
gpt_model = "gpt-3.5-turbo"
perform_search = True
number_of_samples = 1000

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Filter the DataFrame based on the label
df = df[df.label.isin(["true", "mostly-true", "half-true", "barely-true", "false-true", "pants-fire"])]

# If number_of_samples is not None, limit the number of samples
if number_of_samples is not None:
    df = df[:number_of_samples]

# Reset the DataFrame index
df.reset_index(inplace=True)

# Initialize an empty list for predicted labels
predicted_labels = []
reasonings = []

# Initialize the ChatOpenAI model
llm = ChatOpenAI(temperature=0, model=gpt_model, request_timeout=60)

# Iterate over the DataFrame rows
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    statement, author, date = row["statement"], row["author"], row["date"]
    output = fact_check(statement=statement, author=author, date=date, llm=llm, perform_search=perform_search)
    try:
        predicted_label, reasoning = output.split(" -> ")
    except ValueError:
        print(output)
        predicted_label = "error"
        reasoning = "error"
    predicted_labels.append(predicted_label)
    reasonings.append(reasoning)

# Calculate and print accuracy, precision, recall, F1 score, confusion matrix, and classification report
accuracy = accuracy_score(df['label'], predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(df['label'], predicted_labels, average='weighted')
conf_matrix = confusion_matrix(df['label'], predicted_labels)
class_report = classification_report(df['label'], predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix: \n{conf_matrix}")
print(f"Classification Report: \n{class_report}")

# Add the predicted labels as a new column to the DataFrame
df['predicted_label'] = predicted_labels
df['reasoning'] = reasonings

# Construct the DataFrame name
df_name = f"{gpt_model}_{perform_search}_{df.shape[0]}"

# Save the DataFrame to a CSV file
df.to_csv(f"../../../data/results/{df_name}.csv", index=False)

  0%|          | 3/1000 [00:14<1:17:36,  4.67s/it]


KeyboardInterrupt: 

In [11]:
df

,index,author,statement,source,date,label
0,3,Tyler Katzenberger,Says U.S. Sen. Tammy Baldwin “voted to send hu...,National Republican Senatorial Committee,"November 24, 2023",barely-true
1,7,Louis Jacobson,"“Because wages are rising, this Thanksgiving d...",Karine Jean-Pierre,"November 21, 2023",mostly-true
2,15,Jeff Cercone,These 13 books “have been banned from schools ...,Pink,"November 20, 2023",barely-true
3,16,Louis Jacobson,"""Half of American high school students are not...",Dean Phillips,"November 20, 2023",barely-true
4,18,Ciara O'Rourke,Princess Diana “is still alive” and “possibly ...,Viral image,"November 17, 2023",pants-fire
...,...,...,...,...,...,...
995,2455,Tom Kertscher,Says she never tweeted comments linking pedoph...,Kathy Barnette,"May 16, 2022",pants-fire
996,2456,Madison Czopek,“We have 76% of Americans living paycheck to p...,Facebook posts,"May 16, 2022",barely-true
997,2458,Ciara O'Rourke,“Finland joins NATO.”,Facebook posts,"May 16, 2022",barely-true
998,2462,Ciara O'Rourke,“Your government quietly confirmed the fully v...,Viral image,"May 16, 2022",pants-fire


In [4]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 4.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
